In [20]:
#!ls -R
test_dataloader = [()]
for ii,(data,path) in enumerate(test_dataloader):

Variable containing:
 0.3292  1.1415 -2.0191
 0.2631 -1.6492  1.4036
[torch.FloatTensor of size 2x3]

Variable containing:
 0.2986  0.6729  0.0285
 0.2339  0.0346  0.7316
[torch.FloatTensor of size 2x3]



/home/lrl/anaconda3/envs/pytorchpy36/lib/python3.6/site-packages/ipykernel/__main__.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [3]:
import torch as t
from torch.utils import data
from torch.utils.data import DataLoader
c = t.ones(2,2)

In [5]:
c.cuda

<bound method _cuda of 
 1  1
 1  1
[torch.FloatTensor of size 2x2]
>

^C


## 深度学习程序架构
在做深度学习实验或项目时，为了得到最优的模型结果，中间往往需要很多次的尝试和修改。而合理的文件组织结构，以及一些小技巧可以极大地提高代码的易读易用性。根据我的个人经验，在从事大多数深度学习研究时，程序都需要实现以下几个功能：

- 模型定义
- 数据处理和加载
- 训练模型（Train&Validate）
- 训练过程的可视化
- 测试（Test/Inference）

前面提到过，程序主要包含以下功能：

- 模型定义
- 数据加载
- 训练和测试

首先来看程序文件的组织结构：

```
├── checkpoints/
├── data/
│   ├── __init__.py
│   ├── dataset.py
│   └── get_data.sh
├── models/
│   ├── __init__.py
│   ├── AlexNet.py
│   ├── BasicModule.py
│   └── ResNet34.py
└── utils/
│   ├── __init__.py
│   └── visualize.py
├── config.py
├── main.py
├── requirements.txt
├── README.md
```

其中：

- `checkpoints/`： 用于保存训练好的模型，可使程序在异常退出后仍能重新载入模型，恢复训练
- `data/`：数据相关操作，包括数据预处理、dataset实现等
- `models/`：模型定义，可以有多个模型，例如上面的AlexNet和ResNet34，一个模型对应一个文件
- `utils/`：可能用到的工具函数，在本次实验中主要是封装了可视化工具
- `config.py`：配置文件，所有可配置的变量都集中在此，并提供默认值
- `main.py`：主文件，训练和测试程序的入口，可通过不同的命令来指定不同的操作和参数
- `requirements.txt`：程序依赖的第三方库
- `README.md`：提供程序的必要说明

### 关于__init__.py

可以看到，几乎每个文件夹下都有`__init__.py`，一个目录如果包含了`__init__.py` 文件，那么它就变成了一个包（package）。`__init__.py`可以为空，也可以定义包的属性和方法，但其必须存在，其它程序才能从这个目录中导入相应的模块或函数。例如在`data/`文件夹下有`__init__.py`，则在`main.py` 中就可以`from data.dataset import DogCat`。而如果在`__init__.py`中写入`from .dataset import DogCat`，则在main.py中就可以直接写为：`from data import DogCat`，或者`import data; dataset = data.DogCat`，相比于`from data.dataset import DogCat`更加便捷。

### 数据加载

数据的相关处理主要保存在`data/dataset.py`中。关于数据加载的相关操作，在上一章中我们已经提到过，其基本原理就是使用`Dataset`提供数据集的封装，再使用`Dataloader`实现数据并行加载。Kaggle提供的数据包括训练集和测试集，而我们在实际使用中，还需专门从训练集中取出一部分作为验证集。对于这三类数据集，其相应操作也不太一样，而如果专门写三个`Dataset`，则稍显复杂和冗余，因此这里通过加一些判断来区分。对于训练集，我们希望做一些数据增强处理，如随机裁剪、随机翻转、加噪声等，而验证集和测试集则不需要。

数据加载的一些常用库

```python
import os
from PIL import Image
from torch.utils import data
import numpy as np
from torchvision import transforms as T```


```python
import os
from PIL import Image
from torch.utils import data
import numpy as np
from torchvision import transforms as T

class DogCat(data.Dataset):
    
    def __init__(self, root, transforms=None, train=True, test=False):
        '''
        目标：获取所有图片地址，并根据训练、验证（train=True or False）、测试(test=True or False )划分数据
        '''
        self.test = test
        imgs = [os.path.join(root, img) for img in os.listdir(root)] #获取root路径下所有文件或文件夹的名称并将其与root组合起来作为该文件的绝对路径存放                                                  #到imgs列表里

        # test1: data/test1/8973.jpg
        # train: data/train/cat.10004.jpg 
        if self.test:
            imgs = sorted(imgs, key=lambda x: int(x.split('.')[-2].split('/')[-1]))#应该是把x先分割开，再按分割后的数字排序，key是用来进行比较的元素
        else:
            imgs = sorted(imgs, key=lambda x: int(x.split('.')[-2]))
                #train文件的格文件名：'/home/lrl/pytorchlearning/pytorch-book/chapter6-实战指南/data/train/cat.8.jpg'
                #所以实际上split('.')将文件名分成了3部分，而split('.')[-2]就是读取中间那两个.之间的数字，并按这个数字排序
                #而test数据集采用split('.')[-2].split('/')[-1]先按/分割去最后一部分再按.取倒数第二部分
        imgs_num = len(imgs)#获取元素个数
        
        # 划分训练、验证集，验证:训练 = 3:7
        if self.test:
            self.imgs = imgs
        elif train:
            self.imgs = imgs[:int(0.7*imgs_num)]
        else :
            self.imgs = imgs[int(0.7*imgs_num):]            
    
        if transforms is None:
        
            # 数据转换操作，测试验证和训练的数据转换有所区别
	        
            normalize = T.Normalize(mean = [0.485, 0.456, 0.406], 
                                     std = [0.229, 0.224, 0.225])

            # 测试集和验证集
            if self.test or not train: 
                self.transforms = T.Compose([
                    T.Scale(224),
                    T.CenterCrop(224),
                    T.ToTensor(),
                    normalize
                ]) 
            # 训练集
            else :
                self.transforms = T.Compose([
                    T.Scale(256),
                    T.RandomSizedCrop(224),
                    T.RandomHorizontalFlip(),
                    T.ToTensor(),
                    normalize
                ]) 
                
        
    def __getitem__(self, index):
        '''
        返回一张图片的数据
        对于测试集，没有label，返回图片id，如1000.jpg返回1000
        '''
        img_path = self.imgs[index]
        if self.test: 
             label = int(self.imgs[index].split('.')[-2].split('/')[-1])
        else: 
             label = 1 if 'dog' in img_path.split('/')[-1] else 0
        data = Image.open(img_path)
        data = self.transforms(data)
        return data, label
    
    def __len__(self):
        '''
        返回数据集中所有图片的个数
        '''
        return len(self.imgs)
```

### 数据加载部分步骤的总结
- 加载常用库，设置读取图片的类，继承于torch.utils.data.Dataset，主要参数有，数据集路径，判断是训练验证还是测试集，是否进行数据格式转换，
- 构造函数部分，读入图片存储地址并排序后存放到一个列表里（按什么排序？数据没下完之前也不知道）采用spilt函数分割文件名
- 划分训练，验证，测试集
- 设置训练集与验证测试集不同的数据转换方式
- 设置`__getitem__()`返回一张图片的数据：读取列表中存储的地址，分别取出data与label，转换data数据格式，返回data,label
- 设置`__len__()`,返回数据集中所有图片的个数

采用`__getitem__()`加载图片的原因：

DataLoader里面并没有太多的魔法方法，它封装了Python的标准库`multiprocessing`，使其能够实现多进程加速。在此提几点关于Dataset和DataLoader使用方面的建议：
1. 高负载的操作放在`__getitem__`中，如加载图片等。
2. dataset中应尽量只包含只读对象，避免修改任何可变对象，利用多线程进行操作。

第一点是因为多进程会并行的调用`__getitem__`函数，将负载高的放在`__getitem__`函数中能够实现并行加速。
第二点是因为dataloader使用多进程加载，如果在`Dataset`实现中使用了可变对象，可能会有意想不到的冲突。在多线程/多进程中，修改一个可变对象，需要加锁，但是dataloader的设计使得其很难加锁（在实际使用中也应尽量避免锁的存在），因此最好避免在dataset中修改可变对象。例如下面就是一个不好的例子，在多进程处理中`self.num`可能与预期不符，这种问题不会报错，因此难以发现。如果一定要修改可变对象，建议使用Python标准库`Queue`中的相关数据结构。



### 卷积神经网络模型的定义，加载，保存方法
模型的定义主要保存在`models/`目录下，其中`BasicModule`是对`nn.Module`的简易封装，提供快速加载和保存模型的接口

In [5]:
import torch as t
import models
class BasicModule(t.nn.Module):
    '''
    封装了nn.Module，主要提供save和load两个方法
    '''

    def __init__(self):
        super(BasicModule,self).__init__()
        self.model_name = str(type(self)) # 模型的默认名字

    def load(self, path):
        '''
        可加载指定路径的模型
        '''
        self.load_state_dict(t.load(path))

    def save(self, name=None):
        '''
        保存模型，默认使用“模型名字+时间”作为文件名，
        如AlexNet_0710_23:57:29.pth
        '''
        if name is None:
            prefix = 'checkpoints/' + self.model_name + '_'
            name = time.strftime(prefix + '%m%d_%H:%M:%S.pth')
            #Python time strftime() 函数接收以时间元组，并返回以可读字符串表示的当地时间，格式由参数format决定。
        t.save(self.state_dict(), name)
        return name



#### 模型定义，加载，保存，调用的方法

1. 首先像上面那段代码一样定义一个BasicModule的class，继承torch.nn.Module。在这个class里定义模型的加载与保存的方法
2. 其它自定义模型一般继承`BasicModule`，然后实现自己的模型。其中`AlexNet.py`实现了AlexNet，`ResNet34`实现了ResNet34。在 
3. 在`models/__init__py`中里import各卷积神经网络：
    ```python
    from .AlexNet import AlexNet
    from .ResNet34 import ResNet34
    ```
3. 这样在主函数中就可以写成：
     ```python
    from models import AlexNet
    或
    import models
    model = models.AlexNet()
    或
    import models
    model = getattr('models', 'AlexNet')()
    ```

  之后，在实际使用中，直接调用`model.save()`及`model.load(opt.load_path)`即可。

其中最后一种写法最为关键，这意味着我们可以通过字符串直接指定使用的模型，而不必使用判断语句，也不必在每次新增加模型后都修改代码。新增模型后只需要在`models/__init__.py`中加上`from .new_module import new_module`即可。

其它关于模型定义的注意事项，在上一章中已详细讲解，这里就不再赘述，总结起来就是：

- 尽量使用`nn.Sequential`（比如AlexNet）
- 将经常使用的结构封装成子Module（比如GoogLeNet的Inception结构，ResNet的Residual Block结构）
- 将重复且有规律性的结构，用函数生成（比如VGG的多种变体，ResNet多种变体都是由多个重复卷积层组成）


我的问题是什么：
主要是为什么可以调用`model.save()`及`model.load(opt.load_path)`来处理AlexNet类与ResNet34 类，明明这些是 BasicModule类下的函数。
采取的措施：
```python
from models import AlexNet
或
import models
model = models.AlexNet()
或
import models
model = getattr('models', 'AlexNet')()


```__init__文档```
```
from .AlexNet import AlexNet
from .ResNet34 import ResNet34
# from torchvision.models import InceptinV3
# from torchvision.models import alexnet as AlexNe
```
取得的效果：
直接调用`model.save()`及`model.load(opt.load_path)`来保存AlexNet或ResNet34 

原因：BasicModule类是AlexNet()与ResNet34()的父类


### 工具函数

这里采用class Visualizer封装了visdom的基本操作，吃用的时候from utils.visualize import Visualizer即可

### 配置文件
在模型定义、数据处理和训练等过程都有很多变量，这些变量应提供默认值，并统一放置在配置文件中，这样在后期调试、修改代码或迁移程序时会比较方便，在这里我们将所有可配置项放在`config.py`中。

可配置的参数主要包括：

- 数据集参数（文件路径、batch_size等）
- 训练参数（学习率、训练epoch等）
- 模型参数

这样我们在程序中就可以这样使用：

```
import models
from config import DefaultConfig

opt = DefaultConfig()
lr = opt.lr
model = getattr(models, opt.model)
dataset = DogCat(opt.train_data_root)```


这些都只是默认参数，在这里还提供了更新函数，根据字典更新配置参数。

```
python
def parse(self, kwargs):
        '''
        根据字典kwargs 更新 config参数
        '''
        # 更新配置参数
        for k, v in kwargs.items():
            if not hasattr(self, k):
                # 警告还是报错，取决于你个人的喜好
                warnings.warn("Warning: opt has not attribut %s" %k)
            setattr(self, k, v)
            
        # 打印配置信息	
        print('user config:')
        for k, v in self.__class__.__dict__.items():
            if not k.startswith('__'):
                print(k, getattr(self, k))
```

这样我们在实际使用时，并不需要每次都修改`config.py`，只需要通过命令行传入所需参数，覆盖默认配置即可。

例如：

```
opt = DefaultConfig()
new_config = {'lr':0.1,'use_gpu':False}
opt.parse(new_config)
opt.lr == 0.1
```

### train函数的步骤

- 根据命令行参数更新配置（即设定模型，数据加载以及训练时的一些超参数）
- 选取卷积神经网络模型并加载模型与参数
- 用自定义的dataset的子类加载并与处理数据，用dataloader将读取的数据处理成batch
- 设定损失函数和优化器
- 计算重要指标
- 开始训练
  - for epoch in range(opt.max_epoch):
  - 重置loss与混淆矩阵
  - for ii,(data,label) in enumerate(train_dataloader):
  - 将data and label化为变量
  - 若使用gpu，将数据化为.cuda形式
  - 优化器梯度清零
  - 计算输出，loss
  - loss.backward(),optimizer.step()
  - 更新loss存储与混淆矩阵存储
  - loss可视化
- 保存模型
- 计算在验证集上的指标

```python
def train(**kwargs):
    
    # 根据命令行参数更新配置，kwargs 取自命令行参数，如果命令行参数没有就是用config文件里的
    opt.parse(kwargs)
    vis = Visualizer(opt.env)#设置visdom环境
    
    # step1: 定义卷积神经网络模型
    model = getattr(models, opt.model)()#取出与配置参数中的模型名一致的models文件夹下卷积神经网络架构(一般是model.py文件)
    if opt.load_model_path:
        model.load(opt.load_model_path)#这里加载的是存储网络模型各层权重的pth文件
    if opt.use_gpu: model.cuda()

    # step2: 数据
    train_data = DogCat(opt.train_data_root,train=True)#传入训练集地址
    val_data = DogCat(opt.train_data_root,train=False)#传入验证集地址
    train_dataloader = DataLoader(train_data,opt.batch_size,
                        shuffle=True,
                        num_workers=opt.num_workers)#
    val_dataloader = DataLoader(val_data,opt.batch_size,
                        shuffle=False,
                        num_workers=opt.num_workers)
    
    # step3: 目标函数和优化器
    criterion = t.nn.CrossEntropyLoss()
    lr = opt.lr
    optimizer = t.optim.Adam(model.parameters(),
                            lr = lr,
                            weight_decay = opt.weight_decay)
        
    # step4: 统计指标：平滑处理之后的损失，还有混淆矩阵
    loss_meter = meter.AverageValueMeter()
    confusion_matrix = meter.ConfusionMeter(2)
    previous_loss = 1e100

    # 训练
    for epoch in range(opt.max_epoch):
        
        loss_meter.reset()
        confusion_matrix.reset()

        for ii,(data,label) in enumerate(train_dataloader):

            # 训练模型参数 
            input = Variable(data)
            target = Variable(label)
            if opt.use_gpu:
                input = input.cuda()
                target = target.cuda()
            optimizer.zero_grad()
            score = model(input)
            loss = criterion(score,target)
            loss.backward()
            optimizer.step()
            
            # 更新统计指标以及可视化
            loss_meter.add(loss.data[0])
            confusion_matrix.add(score.data, target.data)

            if ii%opt.print_freq==opt.print_freq-1:
                vis.plot('loss', loss_meter.value()[0])
                
                # 如果需要的话，进入debug模式
                if os.path.exists(opt.debug_file):
                    import ipdb;
                    ipdb.set_trace()

        model.save()

        # 计算验证集上的指标及可视化
        val_cm,val_accuracy = val(model,val_dataloader)
        vis.plot('val_accuracy',val_accuracy)
        vis.log("epoch:{epoch},lr:{lr},loss:{loss},train_cm:{train_cm},val_cm:{val_cm}"
        .format(
                    epoch = epoch,
                    loss = loss_meter.value()[0],
                    val_cm = str(val_cm.value()),
                    train_cm=str(confusion_matrix.value()),
                    lr=lr))
        
        # 如果损失不再下降，则降低学习率
        if loss_meter.value()[0] > previous_loss:          
            lr = lr * opt.lr_decay
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
                
        previous_loss = loss_meter.value()[0]
```

### val验证函数的步骤：

验证相对来说比较简单，但要注意需将模型置于验证模式(`model.eval()`)，验证完成后还需要将其置回为训练模式(`model.train()`)，这两句代码会影响`BatchNorm`和`Dropout`等层的运行模式。

- 将模型设置成验证模型：model.eval()
- for ii,(data,label) in enumerate(dataloader):
- 将data and label化为变量(volatile = True 不可求导)
- 若使用gpu，将数据化为.cuda形式
- 计算输出
- 更新混淆矩阵存储
- 把模型恢复为训练模式
- 计算准确率，返回准确率与混淆矩阵

### 测试函数的步骤

测试时，需要计算每个样本属于狗的概率，并将结果保存成csv文件。测试的代码与验证比较相似，但需要自己加载模型和数据。

```python
def test(**kwargs):
    opt.parse(kwargs)
    
    # 模型
    model = getattr(models, opt.model)().eval()
    if opt.load_model_path:
        model.load(opt.load_model_path)
    if opt.use_gpu: model.cuda()

    # 数据
    train_data = DogCat(opt.test_data_root,test=True)
    test_dataloader = DataLoader(train_data,\
							    batch_size=opt.batch_size,\
							    shuffle=False,\
							    num_workers=opt.num_workers)
    
    results = []
    for ii,(data,path) in enumerate(test_dataloader):
        input = t.autograd.Variable(data,volatile = True)
        if opt.use_gpu: input = input.cuda()
        score = model(input)
        probability = t.nn.functional.softmax\
	        (score)[:,1].data.tolist()      
        batch_results = [(path_,probability_) \
	        for path_,probability_ in zip(path,probability) ]
                                   
        results += batch_results
    write_csv(results,opt.result_file)
    return results
```
#这里的path应该是读取的文件（图片）的编号。因为我们在自定义的dataset对象里设置__getitem__函数为如果是训练集返回data and label（根据文件名是dog还是cat）， 如果是测试集就返回data以及该测试文件的索引号，eg1000.jpg就返回1000.
ii,是循环的批号。data是测试集中每个文件的数据，path则是训练集中每个文件的编号

zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表。如果各个迭代器的元素个数不一致，则返回列表长度与最短的对象相同，利用 * 号操作符，可以将元组解压为列表。